In [3]:
# default_exp algo.rs.fm

%reload_ext autoreload
%autoreload 2

# algo-rs-fm
http://litowang.top/2018/07/29/factorization-machine/

FM（factorization machine, 因子分解机）

每一条训练数据都是一条向量x，每条训练数据有n个特征，其中某个特征记为xi，点击率预估模型旨在寻找一个合适的函数f(x)，对于每一个用户-广告pair预测该用户对该广告的点击率，并且返回点击率预估值最高的一个用户-广告pair请求以展示到用户界面。

## 线性模型
$$f(x)=w_{0}+\sum_{i=1}^{n}w_{i}x_{i}$$
缺点: 没有考虑交叉特征。

特征交叉好是好，但是会带来一个麻烦的问题就是维度爆炸，假设原始有10w维的特征，在特征交叉之后特征维度变成了100亿，如此巨大的特征空间对系统内存、通信、计算都有非常大的压力难以训练，如果不对所有的特征交叉进行建模，只对其中部分特征交叉建模是否可行？答案是肯定的，并且业界对此有个专有名词就是特征工程，在LR时代，特征工程最容易提高推荐效果，但是特征交叉的尝试需要试错成本，并且随着特征交叉达到了一定上限之后，试错成本也随着增大，投入产出比愈来愈低。

既然特征交叉这么有用，人工交叉又愈来愈难，有没有办法能在不过分增大参数空间的前提下让模型自动的学习特征交叉权重？（后面可以看到因子分解机系列模型可以比较有效的解决这个问题）

## Poly-2
Poly-2模型相比于LR，多了一个二维的特征穷举交叉项，由此模型可以自动学习每一个特征交叉的权重
$$f(x)=w_{0}+\sum_{i=1}^{n}w_{i}x_{i} +\sum_{i=1}^{n}\sum_{j=1}^{n}w_{ij}x_{i}x_{j}$$
但是该模型存在致命的缺点：

数据稀疏：在稀疏数据中，一个xi已经是很小概率了，而xi和xj同时出现的概率更是微乎其微，为每对特征交叉拟合一个独立参数难以收敛。
参数空间大：参数空间为O(n2)对内存和机器资源和性能要求太高，在实际业务中无法使用。

## 因子分解机模型（FM）
对于Poly-2中的特征交叉部分，可以看做是一个n∗n的矩阵，我们可以借用矩阵论中的矩阵分解法将矩阵分解为两个向量的外积。因此模型的复杂度就随着下降了。

因子分解机就是借助这一方法对特征交叉矩阵进行分解，从而得到一个能有效降维版本的模型，使得模型参数空间减小，模型训练变得可行。FM因子分解机最早由Steffen Rendle提出，该模型公式如下：

$$f(x)=w_{0}+\sum_{i=1}^{n}w_{i}x_{i} +\sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf v_{i},\mathbf v_{j}>x_{i}x_{j}$$
其中$<\mathbf v_{i},\mathbf v_{j}>$
表示的是向量vi 和向量 vj 的内积，如果我们直接第式子中的第三项部分进行计算，复杂度是O(kn2)，简单的对式子进行化简，我们可以得到复杂度更低的计算式O(kn)：
$$\begin{eqnarray}
\sum_{i=1}^{n}\sum_{j=i+1}^{n}<\mathbf v_{i},\mathbf v_{j}>x_{i}x_{j}
&=&\frac{1}{2}\sum_{i=1}^{n}\sum_{j=1}^{n}<\mathbf v_{i},\mathbf v_{j}>x_{i}x_{j} - \frac{1}{2}\sum_{i=1}^{n}<\mathbf v_{i},\mathbf v_{i}>x_{i}x_{i}\\
&=&\frac{1}{2} \left( \sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{f=1}^{k}v_{i,f} \cdot v_{j,f} \cdot x_{i} \cdot x_{j} -\sum_{i=1}^{n}\sum_{f=1}^{k}v_{i,f} \cdot v_{i,f} \cdot x_{i} \cdot x_{i}\right) \\
&=&\frac{1}{2} \left(   \left( \sum_{i=1}^{n} v_{i,f} \cdot x_{i} \right) \left( \sum_{j=1}^{n} v_{j,f} \cdot x_{j} \right) - \sum_{i=1}^{n} v_{i,f}^{2} \cdot x_{i}^{2} \right) \\
&=&\frac{1}{2} \left(   \left( \sum_{i=1}^{n} v_{i,f} \cdot x_{i} \right)^{2} - \sum_{i=1}^{n} v_{i,f}^{2} \cdot x_{i}^{2} \right) \\
\end{eqnarray}$$

In [1]:
#export


# LightFM
https://github.com/lyst/lightfm

https://making.lyst.com/lightfm/docs/home.html

Learning to Rank Sketchfab Models with LightFM 
https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/

LightFM是针对隐式和显式反馈的许多流行推荐算法的Python实现，包括BPR和WARP排名损失的有效实现。 它易于使用，快速（通过多线程模型估计）并产生高质量的结果。

它还使将item和用户元数据都合并到传统的矩阵分解算法中成为可能。 它将每个用户和item表示为其特征的潜在表示的总和，从而允许将recommendations推广到新item（通过item features）和新用户（通过用户features）。


## install

In [5]:
# !pip install lightfm -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip freeze | grep lightfm

lightfm==1.15


## Quickstart
Fitting an implicit feedback model on the MovieLens 100k dataset is very easy:

In [6]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [7]:
# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
data = fetch_movielens(min_rating=5.0)

In [10]:
type(data)

dict

In [11]:
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 19048 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 2153 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}

In [19]:
data['train'].toarray().shape

(943, 1682)

In [20]:
data['train'].toarray()[0]

array([5., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [15]:
# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(data['train'], epochs=30, num_threads=2)

In [31]:
model.get_item_representations()[1].shape

(1682, 10)

In [32]:
len(model.get_user_representations())

2

In [30]:
model.get_user_representations()[1].shape

(943, 10)

In [34]:
model.user_embedding_gradients.shape

(943, 10)

In [35]:
model.predict_rank(data['test'])

<943x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 2153 stored elements in Compressed Sparse Row format>

In [36]:
model.predict_rank(data['test']).shape

(943, 1682)

In [37]:
model.predict_rank(data['test']).toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
model.predict()

In [16]:
# Evaluate the trained model
test_precision = precision_at_k(model, data['test'], k=5).mean()

In [17]:
test_precision

0.051254958

## api

### LightFM

    LightFM(
        no_components=10,
        k=5,
        n=10,
        learning_schedule='adagrad',
        loss='logistic',
        learning_rate=0.05,
        rho=0.95,
        epsilon=1e-06,
        item_alpha=0.0,
        user_alpha=0.0,
        max_sampled=10,
        random_state=None,
    )

A hybrid(混合的) latent representation recommender model.

The model learns embeddings (latent representations in a high-dimensional space) for users and items in a way that encodes user preferences over items.

When multiplied together, these representations produce scores for every item for a given user; items scored highly are more likely to be interesting to the user.

The user and item representations are expressed in terms of representations of their features: an embedding is estimated for every feature, and these features are then summed together to arrive at representations for users and items. 

For example, if the movie 'Wizard of Oz' is described by the following features: 'musical fantasy', 'Judy Garland', and 'Wizard of Oz', then its embedding will be given by taking the features' embeddings and adding them together. The same applies to user features.

The embeddings are learned through `stochastic gradient descent <http://cs231n.github.io/optimization-1/>`_ methods.

Four loss functions are available:

- logistic: useful when both positive (1) and negative (-1) interactions
  are present.
- BPR: Bayesian Personalised Ranking [1]_ pairwise loss. Maximises the prediction difference between a positive example and a randomly chosen negative example. Useful when only positive interactions are present and optimising ROC AUC is desired.
- WARP: Weighted Approximate-Rank Pairwise [2]_ loss. Maximises
  the rank of positive examples by repeatedly sampling negative
  examples until rank violating one is found. Useful when only
  positive interactions are present and optimising the top of
  the recommendation list (precision@k) is desired.
- k-OS WARP: k-th order statistic loss [3]_. A modification of WARP that
  uses the k-th positive example for any given user as a basis for pairwise
  updates.

Two learning rate schedules are available:

- adagrad: [4]_
- adadelta: [5]_

#### Parameters

    no_components: int, optional
        the dimensionality of the feature latent embeddings.
    k: int, optional
        for k-OS training, the k-th positive example will be selected from the
        n positive examples sampled for every user.
    n: int, optional
        for k-OS training, maximum number of positives sampled for each update.
    learning_schedule: string, optional
        one of ('adagrad', 'adadelta').
    loss: string, optional
        one of  ('logistic', 'bpr', 'warp', 'warp-kos'): the loss function.
    learning_rate: float, optional
        initial learning rate for the adagrad learning schedule.
    rho: float, optional
        moving average coefficient for the adadelta learning schedule.
    epsilon: float, optional
        conditioning parameter for the adadelta learning schedule.
    item_alpha: float, optional
        L2 penalty on item features. Tip: setting this number too high can slow
        down training. One good way to check is if the final weights in the
        embeddings turned out to be mostly zero. The same idea applies to
        the user_alpha parameter.
    user_alpha: float, optional
        L2 penalty on user features.
    max_sampled: int, optional
        maximum number of negative samples used during WARP fitting.
        It requires a lot of sampling to find negative triplets for users that
        are already well represented by the model; this can lead to very long
        training times and overfitting. Setting this to a higher number will
        generally lead to longer training times, but may in some cases improve
        accuracy.
    random_state: int seed, RandomState instance, or None
        The seed of the pseudo random number generator to use when shuffling
        the data and initializing the parameters.

#### Attributes

    item_embeddings: np.float32 array of shape [n_item_features, n_components]
         Contains the estimated latent vectors for item features. The [i, j]-th
         entry gives the value of the j-th component for the i-th item feature.
         In the simplest case where the item feature matrix is an identity
         matrix, the i-th row will represent the i-th item latent vector.
    user_embeddings: np.float32 array of shape [n_user_features, n_components]
         Contains the estimated latent vectors for user features. The [i, j]-th
         entry gives the value of the j-th component for the i-th user feature.
         In the simplest case where the user feature matrix is an identity
         matrix, the i-th row will represent the i-th user latent vector.
    item_biases: np.float32 array of shape [n_item_features,]
         Contains the biases for item_features.
    user_biases: np.float32 array of shape [n_user_features,]
         Contains the biases for user_features.

#### Notes

Users' and items' latent representations are expressed in terms of their
features' representations. If no feature matrices are provided to the
:func:`lightfm.LightFM.fit` or :func:`lightfm.LightFM.predict` methods, they are
implicitly assumed to be identity matrices: that is, each user and item
are characterised by one feature that is unique to that user (or item).
In this case, LightFM reduces to a traditional collaborative filtering
matrix factorization method.

When a feature matrix is provided, it should be of shape
``(num_<users/items> x num_features)``. An embedding will then be estimated
for every feature: that is, there will be ``num_features`` embeddings.
To obtain the representation for user i, the model will look up the i-th
row of the feature matrix to find the features with non-zero weights in
that row; the embeddings for these features will then be added together
to arrive at the user representation. For example, if user 10 has weight 1
in the 5th column of the user feature matrix, and weight 3 in the 20th
column, that user's representation will be found by adding together
the embedding for the 5th and the 20th features (multiplying the latter
by 3). The same goes for items.

Note: when supplying feature matrices, an implicit identity feature
matrix will no longer be used. This may result in a less expressive model:
because no per-user features are estiamated, the model may underfit. To
combat this, include per-user (per-item) features (that is, an identity
matrix) as part of the feature matrix you supply.

In [ ]:
from nbdev.export import notebook2script
notebook2script('.ipynb')